In [2]:
from fdh_gallica import Periodical, Search, Document
from fdh_gallica.parallel_process import iiif_urls_for_documents
from tqdm.autonotebook import tqdm
import json
import pandas as pd
from pandas.io.json import json_normalize
import itertools
from itertools import chain
from itertools import compress
import re
import wikipedia
from bs4 import BeautifulSoup
import requests
from SPARQLWrapper import SPARQLWrapper, XML

C:\Users\haeeun\Anaconda3\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# Fetch wikipedia article

In [64]:
def hacked_summary(x):
        
    try:
        x = re.sub('[,(;\-)]', ' ', x)
        keywords = re.findall('[\S]+', x.lower())
        keywords = list(compress(keywords, list(map(lambda x: re.findall('[\d]+[\.\?]+', x.lower()) == [], keywords))))


        p = wikipedia.page(x)

        url = p.url
        s = p.summary
        weight = len(p.images)

        mask = list(map(lambda x: s.lower().find(x) != -1 ,keywords))
        se = list(compress(keywords, mask))

        if len(se) == 0:
            s = ''
            weight = -1
            url = -1
                    
        return [s, list(keywords), list(se), weight, url]
        
    except:
        return ['', [], [],-1,'']

In [ ]:
wiki_names = pd.read_pickle('data/id_name.pkl').names

wiki_reset = False

if wiki_reset:
    wikipedia.set_lang('en')
    wiki_en = pd.DataFrame(wiki_names, columns=['name'])
    wiki_en['result'] = wiki_en.name.apply(hacked_summary)
    wiki_en[['summary','keyword', 'keyword_found','weight','url']] = pd.DataFrame(wiki_en.result.values.tolist(), index= wiki_en.index)
    
    wikipedia.set_lang('fr')
    wiki_fr = pd.DataFrame(wiki_names, columns=['name'])
    wiki_fr['result_fr'] = wiki_fr.name.apply(hacked_summary)
    wiki_fr[['summary_fr','keyword_fr', 'keyword_found_fr','weight_fr', 'url_fr']] = pd.DataFrame(wiki_fr.result_fr.values.tolist(), index= wiki_fr.index)

# takes 2 hours

In [128]:
wiki_en = pd.read_pickle('data/wiki_en.pkl')
wiki_fr = pd.read_pickle('data/wiki_fr.pkl')

In [132]:
wiki_en['len'] = wiki_en.keyword.apply(lambda x: len(x))
wiki_en['len_found'] = wiki_en.keyword_found.apply(lambda x: len(x))
wiki_en['difference'] = wiki_en['len'] - wiki_en['len_found']

wiki_fr['len'] = wiki_fr.keyword_fr.apply(lambda x: len(x))
wiki_fr['len_found'] = wiki_fr.keyword_found_fr.apply(lambda x: len(x))
wiki_fr['difference'] = wiki_fr['len'] - wiki_fr['len_found']

In [89]:
wiki_exclude_en = list(set(wiki_en.name[(wiki_en.len.apply(lambda x: x < 2)) | \
                               (wiki_en.len.apply(lambda x: x in [2,3]) & wiki_en.difference.apply(lambda x: x >0)) | \
                               (wiki_en.len.apply(lambda x: x >3) & wiki_en.difference.apply(lambda x: x >1))]) - \
                  set(['Commerson (1802-1879)', \
                      'Kallérgīs, Dīmī́trios (1803-1867)', 'Sínas, Símōn (1810-1876)', 'Andrée, Ellen (1857-1925)', \
                      'Cain, Auguste Nicolas (1822-1894)' ,'Carmouche, Pierre-Frédéric-Adolphe (1797-1868)', \
                       'Planquette, Robert (1848-1903). [Les voltigeurs de la 32ème]', 'Léopold II (1835-1909 ; roi des Belges)', \
                       
                       'Debussy, Claude (1862-1918)']))

In [156]:
wiki_exclude_fr = list(set(wiki_fr.name[(wiki_fr.len.apply(lambda x: x < 2)) | \
                                    (wiki_fr.len.apply(lambda x: x in [2,3]) & wiki_fr.difference.apply(lambda x: x > 0)) | \
                                    (wiki_fr.len.apply(lambda x: x in [4,5]) & wiki_fr.difference.apply(lambda x: x > 1)) | \
                                    (wiki_fr.len.apply(lambda x: x > 5) & wiki_fr.difference.apply(lambda x: x > 2))]) - \
                       set(['Abbéma, Louise (1858-1927)','Achard, Amédée (1814-1875)',\
                        'Godard, Jules (18..-1... ; aéronaute)', 'Désiré (1823-1873)', \
                        'Kallérgīs, Dīmī́trios (1803-1867)' ,'Sínas, Símōn (1810-1876)','Krauss, Maria Gabriele (1842-1906)', 'Vaillant, Marguerite (1855-19.. ; artiste lyrique)', \
                        'Bloch, Rosine (1832-1891 ; artiste lyrique)', \
                           
                        'Lockroy (1803-1891)','Sicard, Auguste (1839-19..)','Chevé, Émile (1804-1864)','Dubufe, Édouard (1820-1883)'])
                      )

In [157]:
wiki_en_summary = wiki_en[['name','keyword','keyword_found','summary','weight','url']][wiki_en.name.apply(lambda x: x not in wiki_exclude_en)]
wiki_fr_summary = wiki_fr[['name','keyword_fr','keyword_found_fr','summary_fr','weight_fr','url_fr']][wiki_fr.name.apply(lambda x: x not in wiki_exclude_fr)]

In [158]:
#update some info



to_update_en = {'Debussy, Claude (1862-1918)':'Claude Debussy'}
to_update_fr = {'Lockroy (1803-1891)':'Lockroy','Sicard, Auguste (1839-19..)':'Auguste Sicard',\
                'Chevé, Émile (1804-1864)':'Émile-Joseph-Maurice Chevé','Dubufe, Édouard (1820-1883)':'Édouard Dubufe'}

wikipedia.set_lang('en')
for i in to_update_en.items():
    p = wikipedia.page(i[1], auto_suggest=False)
    wiki_en_summary.loc[wiki_en['name'] == i[0], ['summary','weight', 'url']] = [p.content, len(p.images), p.url]

wikipedia.set_lang('fr')
for i in to_update_fr.items():
    p = wikipedia.page(i[1], auto_suggest=False)
    wiki_fr_summary.loc[wiki_fr['name'] == i[0], ['summary_fr','weight_fr', 'url_fr']] = [p.content, len(p.images), p.url]


In [160]:
wiki_en_summary['rank_en'] = wiki_en_summary.weight.rank(ascending=False)
wiki_fr_summary['rank_fr'] = wiki_fr_summary.weight_fr.rank(ascending=False)

In [161]:
wiki_en_summary.sort_values(by='rank_en')
wiki_fr_summary.sort_values(by='rank_fr')

,name,keyword_fr,keyword_found_fr,summary_fr,weight_fr,url_fr,rank_fr
248,"Bouguereau, William (1825-1905)","[bouguereau, william, 1825, 1905]","[bouguereau, william, 1825, 1905]",L'œuvre peint de William Bouguereau (La Rochel...,424,https://fr.wikipedia.org/wiki/%C5%92uvre_peint...,1.0
601,"Dieterle, Amélie (1870-1941)","[dieterle, amélie, 1870, 1941]","[dieterle, amélie, 1941]","Amélie Diéterle, nom de scène d'Amélie Laurent...",129,https://fr.wikipedia.org/wiki/Am%C3%A9lie_Di%C...,2.0
1689,"Sand, George (1804-1876)","[sand, george, 1804, 1876]","[sand, george, 1804, 1876]","George Sand [ ʒɔʁʒ sɑ̃ːd], pseudonyme d'Amanti...",113,https://fr.wikipedia.org/wiki/George_Sand,3.0
1211,"Manet, Édouard (1832-1883)","[manet, édouard, 1832, 1883]","[manet, édouard, 1832, 1883]","Édouard Manet, né le 23 janvier 1832 à Paris e...",105,https://fr.wikipedia.org/wiki/%C3%89douard_Manet,4.0
1628,"Robespierre, Maximilien de (1758-1794)","[robespierre, maximilien, de, 1758, 1794]","[robespierre, maximilien, de, 1758, 1794]","Maximilien de Robespierre, ou Maximilien Robes...",95,https://fr.wikipedia.org/wiki/Maximilien_de_Ro...,5.0
...,...,...,...,...,...,...,...
699,"Expilly, Charles (1814-1886)","[expilly, charles, 1814, 1886]","[expilly, charles, 1814, 1886]",Jean-Charles-Marie Expilly (Salon-de-Provence ...,3,https://fr.wikipedia.org/wiki/Charles_Expilly,989.5
84,"Azimont, Cécile (18..-18..)","[azimont, cécile]","[azimont, cécile]","Cécile Azimont est une actrice française, née ...",3,https://fr.wikipedia.org/wiki/C%C3%A9cile_Azimont,989.5
1857,"Verconsin, Eugène (1823-1891)","[verconsin, eugène, 1823, 1891]","[verconsin, eugène, 1823, 1891]","Eugène Verconsin, né Pierre-Frédéric-Eugène Ve...",3,https://fr.wikipedia.org/wiki/Eug%C3%A8ne_Verc...,989.5
542,"Delahaye, Camille (1842-19..)","[delahaye, camille, 1842]","[delahaye, camille, 1842]",Cette page dresse une liste de personnalités m...,2,https://fr.wikipedia.org/wiki/D%C3%A9c%C3%A8s_...,995.5


209           Bloch, Rosine (1832-1891 ; artiste lyrique)
328     Calzado, Toribio (17..-18.. ; Directeur du thé...
338                Carlos (1848-1909 ; prince de Bourbon)
446          Conny, Julien-Édouard (1818-1900 ; baron de)
468                        Courcy, Charles de (1836-1917)
490             Daguerre, Louis-Jacques-Mandé (1787-1851)
531       Decazes, Louis-Charles-Elie-Amanieu (1819-1886)
569        Dereims, Etienne (1845-1904 ; artiste lyrique)
665                       Dupray, Henry-Louis (1841-1909)
749           Franck, Paul (18..-19.. ; homme de théâtre)
755     Freycinet, Charles Louis de Saulces de (1828-1...
787                 Garnier, Philippe Étienne (1861-1925)
823                  Glaize, Pierre Paul Léon (1842-1932)
935             Hoyos-Sprinzenstein, Ladislas (1834-1901)
959             Isabelle II (1830-1904 ; reine d'Espagne)
1024                   Krauss, Maria Gabriele (1842-1906)
1098         Lavieille, Eugène-Antoine-Samuel (1820-1889)
1115          

In [ ]:
wiki_en_summary.to_pickle('data/wiki_en_summaries.pkl')
wiki_fr_summary.to_pickle('data/wiki_fr_summaries.pkl')

In [78]:
'''mask = wiki_en.len.apply(lambda x: x < 2)
mask = wiki_en.len.apply(lambda x: x in [2,3]) & wiki_en.difference.apply(lambda x: x >0)
mask = wiki_en.len.apply(lambda x: x >3) & wiki_en.difference.apply(lambda x: x >1)



mask = wiki_en.len.apply(lambda x: x == 3) & wiki_en.difference.apply(lambda x: x == 1)
# except 'Commerson (1802-1879)'
mask = wiki_en.len.apply(lambda x: x == 3) & wiki_en.difference.apply(lambda x: x == 2)
mask = wiki_en.len.apply(lambda x: x == 4) & wiki_en.difference.apply(lambda x: x == 2)
# excpet 'Debussy, Claude (1862-1918)'
# 'Kallérgīs, Dīmī́trios (1803-1867)' 'Sínas, Símōn (1810-1876)' 'Andrée, Ellen (1857-1925)'
mask = wiki_en.len.apply(lambda x: x > 4) & wiki_en.difference.apply(lambda x: x >1)
# 'Cain, Auguste Nicolas (1822-1894)' ,'Carmouche, Pierre-Frédéric-Adolphe (1797-1868)', Planquette, Robert (1848-1903). [Les voltigeurs de la 32ème]'
# 'Léopold II (1835-1909 ; roi des Belges)'

#wiki_en[mask]'''


#mask = wiki_en.len_found.apply(lambda x: x  == 1)
# 'Abbéma, Louise (1858-1927)','Achard, Amédée (1814-1875)'

mask = wiki_fr.len.apply(lambda x: x < 2)
mask = wiki_fr.len.apply(lambda x: x in [2,3]) & wiki_fr.difference.apply(lambda x: x > 0)
mask = wiki_fr.len.apply(lambda x: x in [4.5]) & wiki_fr.difference.apply(lambda x: x > 1)
mask = wiki_fr.len.apply(lambda x: x > 5) & wiki_fr.difference.apply(lambda x: x > 2)



mask = wiki_fr.len.apply(lambda x: x == 3) & wiki_fr.difference.apply(lambda x: x == 1)
# 'Godard, Jules (18..-1... ; aéronaute)' 'Désiré (1823-1873)'
# update: Sicard, Auguste (1839-19..) https://fr.wikipedia.org/wiki/Auguste_Sicard

mask = wiki_fr.len.apply(lambda x: x == 3) & wiki_fr.difference.apply(lambda x: x == 2)
#updata lockroy

mask = wiki_fr.len.apply(lambda x: x == 4) & wiki_fr.difference.apply(lambda x: x == 2)
#update 'Chevé, Émile (1804-1864)', Dubufe, Édouard (1820-1883) https://fr.wikipedia.org/wiki/%C3%89douard_Dubufe
# 'Kallérgīs, Dīmī́trios (1803-1867)' ,'Sínas, Símōn (1810-1876)'


mask = wiki_fr.len.apply(lambda x: x == 5) & wiki_fr.difference.apply(lambda x: x == 2)
# 'Krauss, Maria Gabriele (1842-1906)', 'Vaillant, Marguerite (1855-19.. ; artiste lyrique)'

mask = wiki_fr.len.apply(lambda x: x == 5) & wiki_fr.difference.apply(lambda x: x == 3)

#mask = wiki_fr.len.apply(lambda x: x > 5) & wiki_fr.difference.apply(lambda x: x >2)
#'Bloch, Rosine (1832-1891 ; artiste lyrique)'



wiki_fr[mask]

,name,summary_fr,keyword_fr,keyword_found_fr,weight_fr,url_fr,len,len_found,difference
468,"Courcy, Charles de (1836-1917)","Courcy est une commune française, située dans ...","[courcy, charles, de, 1836, 1917]","[courcy, de]",30,https://fr.wikipedia.org/wiki/Courcy_(Marne),5,2,3
1147,"Leroux, Marie, Joséphine, Laurence (1843-19..)","George Sand [ ʒɔʁʒ sɑ̃ːd], pseudonyme d'Amanti...","[leroux, marie, joséphine, laurence, 1843]","[marie, 1843]",113,https://fr.wikipedia.org/wiki/George_Sand,5,2,3
1511,"Place, Charles Philippe (1814-1893)",Charles-Philippe de France comte d'Artois plus...,"[place, charles, philippe, 1814, 1893]","[charles, philippe]",39,https://fr.wikipedia.org/wiki/Charles_X,5,2,3
